In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

from scipy.sparse import find, csr_matrix
from pandas.api.types import CategoricalDtype

# Data Loading & Clearing

In [2]:
#Ideas

#How could user enter book: Enter part of title, then search in books dataframe with .title.str.contains(<input>), 
#then sort results by number of ratings to get the most rated book and hence the highest chance of good recommendations.

#Also, filter the outputs not to contain the same string which was given as input

In [3]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore') #There are some suspicious ISBNs, we might loos them as they will not match with books on merging
ratings.rename(columns={'User-ID':'user'
                        , 'Book-Rating': 'rating'}, inplace=True)
ratings.shape
ratings.nunique()

(1149780, 3)

user      105283
ISBN      340553
rating        11
dtype: int64

In [4]:
# dtype_mapping = {
#     'Year-Of-Publication': float
# }
books = pd.read_csv('BX-Books.csv', sep = ';', encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')

#books.drop(books[books['Unnamed: 8'].notna()].index, axis = 0, inplace=True) #Dropping the shifted rows. There is 19 of them, not worth the manual correction I think.
#books.drop(['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'], axis = 1, inplace=True)
books.rename(columns={'Book-Title':'title'
                        , 'Book-Author': 'author'
                        , 'Year-Of-Publication': 'year'
                        , 'Publisher': 'publisher'}, inplace=True)
#books
books.nunique()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
d:\Program_files\Anaconda_nw_install_2022\lib\site-packages\IPython\core\i

ISBN           271360
title          242135
author         102023
year              202
publisher       16807
Image-URL-S    271044
Image-URL-M    271044
Image-URL-L    271041
dtype: int64

In [5]:
users = pd.read_csv('BX-Users.csv', sep = ';' , encoding='utf-8', on_bad_lines='warn', encoding_errors = 'ignore')
users.rename(columns={'User-ID':'user'
                        , 'Location': 'location'
                        , 'Age': 'age'}, inplace=True)
print(users.shape)
users.nunique()

(278858, 3)


user        278858
location     57293
age            165
dtype: int64

In [6]:
ratings

,user,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [7]:
#how many ratings each book have?
explicit_rating_counts = ratings[ratings['rating'] > 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().rename(columns={'rating':'explicit_ratings'})
implicit_rating_counts = ratings[ratings['rating'] == 0][['ISBN', 'rating']].groupby('ISBN', as_index=False).count().rename(columns={'rating':'implicit_ratings'})
#implicit_rating_counts

if 'explicit_ratings' not in books.columns:
    books = books.merge(explicit_rating_counts, on = 'ISBN', how='left')
    books = books.merge(implicit_rating_counts, on = 'ISBN', how='left')

In [140]:
#Clearing the years of publication
books.drop(books[books['year'] == 'Gallimard'].index, axis = 0, inplace=True)
books.drop(books[books['year'] == 'DK Publishing Inc'].index, axis = 0, inplace=True)
books.loc[books.year > 2023, 'year'] = 0 #setting impossible years to 0
books['year'] = books['year'].astype(int, errors='ignore')

#books.loc[books.year == 0, 'year'] = np.NAN ## <<------- this forces the column to be floats. So I will let the zeros stay for now, see how it will affect the rest.

In [141]:
books.year.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984,    0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 1911,
       1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 1934, 1910, 1933,
       1902, 1924, 1921, 1900, 1944, 1917, 1901, 2010, 1908, 1906, 1935,
       1806, 2021, 2012, 2006, 1909, 2008, 1378, 1919, 1922, 1897, 1376])

In [11]:
#books['year'] = books['year'].astype(int, errors='ignore')
books['year'].median()

1995.0

In [12]:
#Adding a time period when the book was released, instead of a year
#The idea here is to add firslty the tens of the book release, and secondly the most adjecent tens as a secondary variable
#Hence e.g. for 1991 -> 1990 and 1980, and for 1998 -> 1990 and 2000
books['period1'] = (books.year // 10) * 10
books['period2'] = books['period1'] + ((((books.year - books.period1) >= 5)*2-1)*10)


In [13]:
#books[['ISBN', 'year', 'period1', 'period2']]

# Bibliographic Similarity Selection

In [222]:
##Inputs/argumets
# input_name = 'The Lord of the Rings' #Unclear input, when name of the series is given, instead of a name of a book
input_name = 'The Fellowship of the Ring'
# input_name = 'The Eye of the World'
# input_name = 'Great Expectations' 
# input_name = 'The Color of Magic'
# input_name = 'The Egyptian' #Here is a problem that title contains 'A Novel'. Giving more weight to author could help?
# input_name = 'Hotel New Hampshire'
# input_name = 'The Shining'
# input_name = 'For Whom the Bell Tolls'

input_book = (books[(books['title'].str.contains(input_name))
                .replace(np.NaN, False)]
                .sort_values(['explicit_ratings', 'implicit_ratings'], ascending = False).head(1)[['ISBN', 'title', 'author', 'year', 'publisher', 
                                                                                                   'explicit_ratings', 'implicit_ratings',
                                                                                                    'period1', 'period2']]
                )

display(input_book)

input_index = input_book.index[0]
input_ISBN = input_book.ISBN.values[0]
              
choose_n = 5

,ISBN,title,author,year,publisher,explicit_ratings,implicit_ratings,period1,period2
1571,0345339703,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,131.0,126.0,1980,1990


In [223]:
#books[(books['author'].str.contains('Tolkien')).replace(np.NaN, False)]
#books[(books['author'].str.contains('Robert Jordan')).replace(np.NaN, False)].head(10)
#books[(books['title'].str.contains('The Eye of the World')).replace(np.NaN, False)]

In [224]:
books['info'] = ( books.title + ' ' +
                 (books.author.fillna('') + ' ')*1 
                 + books.publisher.fillna('') + ' ' 
                 + books.year.astype(str) + ' ' 
                 + books.period1.astype(str) + ' ' 
                 + books.period2.astype(str))
#Here possibly add weight by multiplying the number of times the info appears?
#But if I will use cosine similarity, this way of adding weights is probably useless (?): https://www.pinecone.io/learn/vector-similarity/

In [225]:
books[books['info'].isna()] #empty df
for i in books['info'][0:3]:
    print(i)

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info


Classical Mythology Mark P. O. Morford Oxford University Press 2002 2000 1990
Clara Callan Richard Bruce Wright HarperFlamingo Canada 2001 2000 1990
Decision in Normandy Carlo D'Este HarperPerennial 1991 1990 1980


In [226]:
vectorizer = CountVectorizer()
term_matrix = vectorizer.fit_transform(books['info'])

In [227]:
len(books)
term_matrix.shape
#print(term_matrix[0:5])

271357

(271357, 117284)

In [228]:
#similarities = cosine_similarity(term_matrix, dense_output=False)

In [229]:
#Computing similarities of all books takes too long. Perhaps I could select one book and compute similarity of it with the remaining.

In [230]:
row_to_compare = term_matrix[input_index, :]
similarities = cosine_similarity(row_to_compare, term_matrix)

In [231]:
similarities
similarities.shape

#print(np.sort(similarities)[:,-10:-1])
#print(similarities.sort())

indices_of_largest = np.argsort(similarities)[:, -(100+1):-1]
indices_of_largest = np.flip(indices_of_largest.flatten())
#indices_of_largest

array([[0.05679618, 0.05679618, 0.11973687, ..., 0.53881591, 0.05679618,
        0.27322953]])

(1, 271357)

In [232]:
similarity_selection = books.iloc[indices_of_largest.flatten()].copy()
similarity_selection[~(similarity_selection.title.str.contains(input_name))].iloc[0:choose_n]

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info
4206,0345339738,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,77.0,96.0,1980,1990,"The Return of the King (The Lord of the Rings,..."
780,0345339711,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,83.0,94.0,1980,1990,"The Two Towers (The Lord of the Rings, Part 2)..."
6866,0345340426,The Hobbit and The Lord of the Rings,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345340426.0...,http://images.amazon.com/images/P/0345340426.0...,http://images.amazon.com/images/P/0345340426.0...,6.0,2.0,1980,1990,The Hobbit and The Lord of the Rings J.R.R. TO...
3354,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,161.0,120.0,1980,1990,The Hobbit : The Enchanting Prelude to The Lor...
135872,039556008X,The War of the Ring (The History of The Lord o...,J.R.R. Tolkien,1990,Houghton Mifflin,http://images.amazon.com/images/P/039556008X.0...,http://images.amazon.com/images/P/039556008X.0...,http://images.amazon.com/images/P/039556008X.0...,1.0,NaN,1990,1980,The War of the Ring (The History of The Lord o...


# Explicit Ratings Selection

In [233]:
explicit_ratings = ratings[ratings.rating != 0].copy()
implicit_ratings = ratings[ratings.rating == 0].copy()
implicit_ratings.rating =+ 1

In [234]:
explicit_ratings.rating.value_counts().sort_index() #users mostly rate 5 and higher
#Potentially it might be useful to roup ratings into e.g. 4 clases: 1 to 4, 5+6, 7+8, 9+10

1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: rating, dtype: int64

In [235]:
#Converting explicit ratings into pivotted form in a sparse matrix (pd.pivot() cannot produce sparse it seems)
#kudos to this post: https://stackoverflow.com/questions/31661604/efficiently-create-sparse-pivot-tables-in-pandas
ISBN_c = CategoricalDtype(explicit_ratings.ISBN.unique(), ordered=False)
user_c = CategoricalDtype(explicit_ratings.user.unique(), ordered=False)

row = explicit_ratings.ISBN.astype(ISBN_c).cat.codes
col = explicit_ratings.user.astype(user_c).cat.codes
explicit_sparse_matrix = csr_matrix((explicit_ratings["rating"], (row, col)), 
                           shape=(ISBN_c.categories.size, user_c.categories.size),
                           dtype=int)

explicit_sparse_matrix
expl_index = ISBN_c.categories.get_loc(input_ISBN)

<185972x77805 sparse matrix of type '<class 'numpy.intc'>'
	with 433671 stored elements in Compressed Sparse Row format>

## Explicit Ratings - Cosine similarity

In [236]:
#Nearest Neighbors search
model = NearestNeighbors(n_neighbors = choose_n+1, algorithm = 'auto', metric='cosine')
model.fit(explicit_sparse_matrix)
distances, suggestions = model.kneighbors(explicit_sparse_matrix[expl_index,:], return_distance = True)
#Euclidean distance here does not work, because it takes 0s into consideration, so it outputs random books
#Cosine distace works, but does not take into consideration the size of vector elements - the reviews themselves, so in the end it 
#is the same as implicit rating.
#I was also thinking about a custom distance metric based on euclidean, but dividing the distance by # of common non-zero obs., 
#but didnt finished the idea
#I will try alternative where I will keep only users (columns) that rated input_book

NearestNeighbors(metric='cosine', n_neighbors=6)

In [237]:
explicit_ISBN_suggestions = ISBN_c.categories[list(suggestions[0])]
books[books.ISBN.isin(explicit_ISBN_suggestions)]

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info
780,0345339711,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,83.0,94.0,1980,1990,"The Two Towers (The Lord of the Rings, Part 2)..."
1571,0345339703,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,131.0,126.0,1980,1990,The Fellowship of the Ring (The Lord of the Ri...
3354,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,161.0,120.0,1980,1990,The Hobbit : The Enchanting Prelude to The Lor...
3597,068485080X,DIANA HER TRUE STORY COMMEMORATIVE EDITION,Andrew Morton,1997,Simon &amp; Schuster,http://images.amazon.com/images/P/068485080X.0...,http://images.amazon.com/images/P/068485080X.0...,http://images.amazon.com/images/P/068485080X.0...,6.0,9.0,1990,2000,DIANA HER TRUE STORY COMMEMORATIVE EDITION And...
4206,0345339738,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,77.0,96.0,1980,1990,"The Return of the King (The Lord of the Rings,..."
210118,0688163424,To Timbuktu,Mark Jenkins,1998,Quill,http://images.amazon.com/images/P/0688163424.0...,http://images.amazon.com/images/P/0688163424.0...,http://images.amazon.com/images/P/0688163424.0...,2.0,1.0,1990,2000,To Timbuktu Mark Jenkins Quill 1998 1990 2000


## Explicit Ratings - Euclidean distance on common

In [238]:
_, col_indices, _ = find(explicit_sparse_matrix[expl_index,:])

#Create new sparse matrix with the users that rated input book
subset_sparse_matrix = explicit_sparse_matrix[:, col_indices].copy()

In [239]:
subset_sparse_matrix[expl_index,:]

<1x131 sparse matrix of type '<class 'numpy.intc'>'
	with 131 stored elements in Compressed Sparse Row format>

In [240]:
model = NearestNeighbors(n_neighbors = choose_n+1, algorithm = 'auto', metric='cosine')
model.fit(subset_sparse_matrix)
distances, suggestions = model.kneighbors(subset_sparse_matrix[expl_index,:], return_distance = True)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [241]:
explicit_ISBN_suggestions = ISBN_c.categories[list(suggestions[0])]
books[books.ISBN.isin(explicit_ISBN_suggestions)]

,ISBN,title,author,year,publisher,Image-URL-S,Image-URL-M,Image-URL-L,explicit_ratings,implicit_ratings,period1,period2,info
780,0345339711,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,http://images.amazon.com/images/P/0345339711.0...,83.0,94.0,1980,1990,"The Two Towers (The Lord of the Rings, Part 2)..."
1571,0345339703,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,http://images.amazon.com/images/P/0345339703.0...,131.0,126.0,1980,1990,The Fellowship of the Ring (The Lord of the Ri...
2143,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...,http://images.amazon.com/images/P/059035342X.0...,313.0,258.0,1990,2000,Harry Potter and the Sorcerer's Stone (Harry P...
3354,0345339681,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,http://images.amazon.com/images/P/0345339681.0...,161.0,120.0,1980,1990,The Hobbit : The Enchanting Prelude to The Lor...
3459,0439064872,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,http://images.amazon.com/images/P/0439064872.0...,189.0,162.0,2000,1990,Harry Potter and the Chamber of Secrets (Book ...
4206,0345339738,"The Return of the King (The Lord of the Rings,...",J.R.R. TOLKIEN,1986,Del Rey,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,http://images.amazon.com/images/P/0345339738.0...,77.0,96.0,1980,1990,"The Return of the King (The Lord of the Rings,..."


# Implicit Ratings Selection